In [1]:
# !pip install gensim --user

In [10]:
from tqdm.notebook import tqdm
import pandas as pd

In [11]:
!ls ../HinDroid/data/processed

ls: ../HinDroid/data/processed: No such file or directory


In [12]:
import numpy as np
from scipy import sparse

In [13]:
A = sparse.load_npz('../HinDroid/data/processed/A.npz')
B = sparse.load_npz('../HinDroid/data/processed/B.npz')
P = sparse.load_npz('../HinDroid/data/processed/P.npz')
A_csr = A
A_csc = A.tocsc(copy=True)  # memory is cheap ;D

FileNotFoundError: [Errno 2] No such file or directory: '../HinDroid/data/processed/A.npz'

In [5]:
A.shape

(965, 1545196)

## Sample paths strictly using metapath

In [8]:
# ABPBA
def iter_neighbors():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_bi = np.random.choice(np.nonzero(B[:, api_i])[0])
        api_p = np.random.choice(np.nonzero(P[:, api_bi])[0])
        api_bj = np.random.choice(np.nonzero(B[:, api_p])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_bj])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} api_{api_p} api_{api_bj} app_{app_j}'
        
it = iter_neighbors()

In [9]:
# APA
def iter_neighbors():
    while True:
        app_i = np.random.choice(np.arange(A.shape[0]))
        api_i = np.random.choice(np.nonzero(A_csr[app_i])[1])
        api_p = np.random.choice(np.nonzero(P[:, api_i])[0])
        app_j = np.random.choice(np.nonzero(A_csc[:, api_p])[0])

        yield f'app_{app_i} api_{api_i} api_{api_p} app_{app_j}'
        
it = iter_neighbors()

## Sample paths strictly using node2vec

In [6]:
def get_api_neighbors_A(app):
    assert app.startswith('app_')
    app_id = int(app.split('_')[1])
    neighbor_ids = np.nonzero(A_csr[app_id])[1]
    return np.array([f'api_{s}' for s in neighbor_ids])

def get_app_neighbors_A(api):
    assert api.startswith('api_')
    api_id = int(api.split('_')[1])
    neighbor_ids = np.nonzero(A_csc[:, api_id])[0]
    return np.array([f'app_{s}' for s in neighbor_ids])

def get_api_neighbors_B(api):
    assert api.startswith('api_')
    api_id = int(api.split('_')[1])
    neighbor_ids = np.nonzero(B[:, api_id])[0]
    ls = [f'api_{s}' for s in neighbor_ids]
    ls.remove(api)
    return np.array(ls)

def get_api_neighbors_P(api):
    assert api.startswith('api_')
    api_id = int(api.split('_')[1])
    neighbor_ids = np.nonzero(P[:, api_id])[0]
    ls = [f'api_{s}' for s in neighbor_ids]
    ls.remove(api)
    return np.array(ls)

def all_neighbors_from_api(api):
    assert api.startswith('api_')
    api_id = int(api.split('_')[1])
    nbr_apis = np.concatenate([
        get_api_neighbors_B(api),
        get_api_neighbors_P(api)
    ])
    nbr_apis = np.unique(nbr_apis)
    nbr_apps = get_app_neighbors_A(api)
    # weights later?
    return nbr_apis, nbr_apps

In [7]:
all_neighbors_from_api('api_123481')

NameError: name 'np' is not defined

In [8]:
def node2vec(p, q, walk_length):
    path = []
    
    app = 'app_' + str(np.random.choice(np.arange(A.shape[0])))
    prev_nbrs = get_api_neighbors_A(app)
    curr_node = np.random.choice(prev_nbrs)
    prev_node = app
    path.append(app)
    path.append(curr_node)
    
    for i in range(walk_length - 2):
        if curr_node.startswith('api_'):
            nbr_apis, nbr_apps = all_neighbors_from_api(curr_node)
            curr_nbrs = np.concatenate([nbr_apis, nbr_apps])
        elif curr_node.startswith('app_'):
            curr_nbrs = get_api_neighbors_A(curr_node)
        else: raise AssertionError

        alpha_1 = np.intersect1d(prev_nbrs, curr_nbrs, assume_unique=True)
        alpha_p = prev_node
        alpha_q = np.setdiff1d(np.setdiff1d(curr_nbrs, alpha_1, assume_unique=True), [alpha_p], assume_unique=True)
        alphas = [*alpha_1, alpha_p, *alpha_q]
        assert len(alpha_1) + len(alpha_q) + 1 == len(curr_nbrs)

        probs_q = np.full(len(alpha_q), 1/q/len(alpha_q)) if len(alpha_q) else []
        probs_1 = np.full(len(alpha_1), 1/len(alpha_1)) if len(alpha_1) else []
        probs = [*probs_1, 1/p, *probs_q]
        probs = np.array(probs) / sum(probs)

        new_node = np.random.choice(alphas, p=probs)
        path.append(new_node)
        prev_node = curr_node
        prev_nbrs = curr_nbrs
        curr_node = new_node

    return path

In [9]:
node2vec(p=2, q=1, walk_length=20)

NameError: name 'np' is not defined

In [17]:
f = open('test.cor', 'w')
for _ in tqdm(range(50000)):
    f.write(next(it) + '\n')
f.close()

In [18]:
!head test.cor

app_143 api_223845 api_1175 api_518178 api_58119 app_400
app_420 api_912739 api_912741 api_912704 api_511 app_89
app_184 api_31956 api_31954 api_31956 api_31956 app_659
app_96 api_615044 api_399 api_18131 api_112268 app_86
app_88 api_198180 api_9452 api_31406 api_8794 app_370
app_503 api_1378901 api_1378890 api_1378881 api_1245 app_650
app_603 api_1422894 api_1422926 api_1422933 api_1378833 app_776
app_629 api_203149 api_203149 api_203148 api_203162 app_363
app_912 api_12597 api_5907 api_5907 api_343452 app_39
app_369 api_156989 api_155563 api_155563 api_156957 app_144


In [20]:
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'test.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield line.strip().split(' ')

In [21]:
sentences = MyCorpus()

In [25]:
%%time

import gensim.models

sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, size=100, window=3, min_count=3)

CPU times: user 7.17 s, sys: 159 ms, total: 7.33 s
Wall time: 4.96 s


In [26]:
model.wv.similarity('app_657', 'app_942')


0.999475

In [24]:
A.shape

(965, 1545196)

In [28]:
model['app_942']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.04796958,  0.23702888,  0.08210362, -0.06751843, -0.07989473,
       -0.01211152, -0.17681502, -0.27078804,  0.18759327, -0.2571644 ,
       -0.09646761, -0.26214457, -0.04355191,  0.10339639, -0.00111206,
       -0.04970274, -0.2333289 , -0.02887182,  0.00839954,  0.18583256,
        0.08310779,  0.20854019, -0.13642798,  0.08856378,  0.11460141,
       -0.05970431, -0.0396808 , -0.0580118 ,  0.1861048 ,  0.14166604,
        0.18874685, -0.09615654, -0.03615683, -0.3191314 ,  0.00710284,
        0.1492034 , -0.19152488,  0.09995519, -0.14107478, -0.12667376,
        0.02753225,  0.0782541 , -0.02158086, -0.02293486,  0.12774907,
       -0.06648286,  0.04882701, -0.13641298,  0.29304755,  0.05630458,
       -0.1800686 ,  0.01412215, -0.36416933, -0.01393446, -0.172169  ,
       -0.14894058,  0.01741204, -0.07601491, -0.20409897, -0.03510374,
        0.1341842 , -0.14132942, -0.00722764,  0.3104902 ,  0.22619513,
       -0.04606285, -0.02647428, -0.30913672, -0.2217428 , -0.01

In [27]:
model['app_657']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.03690397,  0.1519997 ,  0.05251487, -0.04614409, -0.04838946,
       -0.01040604, -0.11856149, -0.1755112 ,  0.13031587, -0.16470975,
       -0.06813024, -0.17043822, -0.03154248,  0.06752912,  0.00152061,
       -0.02572533, -0.15746999, -0.015014  ,  0.01024781,  0.11990374,
        0.05260967,  0.13818143, -0.0874763 ,  0.06150728,  0.07272373,
       -0.0389541 , -0.02107294, -0.04206243,  0.1261849 ,  0.09562762,
        0.1254481 , -0.06253088, -0.02087012, -0.21542546,  0.00782157,
        0.10394286, -0.1278079 ,  0.06948798, -0.10173127, -0.08416139,
        0.01708622,  0.04923258, -0.01016395, -0.01648844,  0.0864317 ,
       -0.04355844,  0.03417494, -0.09163509,  0.1959702 ,  0.03707016,
       -0.11737441,  0.01009676, -0.23544325, -0.0167006 , -0.11862648,
       -0.0954601 ,  0.01415366, -0.05194201, -0.1378135 , -0.02366411,
        0.09060953, -0.09222126, -0.01168453,  0.20484355,  0.14801997,
       -0.02640378, -0.01316618, -0.20634326, -0.14547949, -0.01

In [84]:
sims = model.wv.most_similar(positive=['app_942'], topn=1000)
[i for i in sims if i[0].startswith('app')]

[('app_433', 0.9917212724685669),
 ('app_912', 0.99168860912323),
 ('app_624', 0.9913800358772278),
 ('app_389', 0.9912938475608826),
 ('app_688', 0.9893128871917725),
 ('app_459', 0.9890421032905579),
 ('app_490', 0.9887189269065857),
 ('app_837', 0.9879517555236816),
 ('app_708', 0.9864774346351624),
 ('app_425', 0.9853894114494324),
 ('app_304', 0.985337495803833),
 ('app_844', 0.9851529598236084),
 ('app_422', 0.9847480058670044),
 ('app_618', 0.9846534132957458),
 ('app_460', 0.9846526384353638),
 ('app_87', 0.9845632910728455),
 ('app_668', 0.9842556715011597),
 ('app_109', 0.9842094779014587),
 ('app_145', 0.9841811656951904),
 ('app_143', 0.9839550256729126),
 ('app_846', 0.9826507568359375),
 ('app_791', 0.9824235439300537),
 ('app_40', 0.9816844463348389),
 ('app_804', 0.9812094569206238),
 ('app_888', 0.9811477661132812),
 ('app_675', 0.9810740947723389),
 ('app_889', 0.980964720249176),
 ('app_106', 0.9809262156486511),
 ('app_146', 0.9805666208267212),
 ('app_345', 0.98040

In [76]:
[i for i in model.wv.vocab.keys() if i.startswith('api')]

['api_1374204',
 'api_1374222',
 'api_16178',
 'api_16184',
 'api_7766',
 'api_30832',
 'api_106552',
 'api_12367',
 'api_1447632',
 'api_11727',
 'api_38844',
 'api_5706',
 'api_8290',
 'api_565807',
 'api_1422788',
 'api_1422800',
 'api_12369',
 'api_12378',
 'api_64193',
 'api_14221',
 'api_5996',
 'api_5995',
 'api_2541',
 'api_54583',
 'api_12618',
 'api_5774',
 'api_18269',
 'api_18470',
 'api_15904',
 'api_15810',
 'api_1455496',
 'api_511',
 'api_5224',
 'api_10999',
 'api_11004',
 'api_1805',
 'api_1835',
 'api_977003',
 'api_7559',
 'api_29590',
 'api_10137',
 'api_45044',
 'api_119459',
 'api_9192',
 'api_14536',
 'api_1575',
 'api_11656',
 'api_1374165',
 'api_1374520',
 'api_15365',
 'api_708925',
 'api_1374225',
 'api_1378934',
 'api_1160153',
 'api_1512347',
 'api_14566',
 'api_105',
 'api_148224',
 'api_36',
 'api_1898',
 'api_1393233',
 'api_1374518',
 'api_1374176',
 'api_193999',
 'api_245',
 'api_1372843',
 'api_1372844',
 'api_390',
 'api_661',
 'api_12005',
 'api_

In [12]:
from gensim.models.word2vec import FAST_VERSION

In [13]:
FAST_VERSION

0

In [16]:
!pip install cython

Defaulting to user installation because normal site-packages is not writeable
